In [25]:
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
import ray

# Load the data
house_price_data = pd.read_csv('House_Price_Data.csv')
    # Convert date string to a numeric feature, e.g., days since a reference date
    # Here, assuming 'date' is the column with date strings like '20140527T000000'
house_price_data['date'] = pd.to_datetime(house_price_data['date']).map(pd.Timestamp.toordinal)

# Create the feature matrix X and target variable y
X = house_price_data[['Area', 'Bedrooms', 'Location', 'Condition', 'Ybuilt', 'Grade', 'Floors', 'View', 'Waterfront', 'sqft_living', 'yr_renovated', 'sqft_basement', 'sqft_above', 'sqft_living15', 'sqft_lot15', 'date']]
y = house_price_data['Price']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Shutdown Ray
ray.shutdown()
ray.init(num_cpus=12)
# Initialize Ray

@ray.remote
def train_neural_network(X_train_part, y_train_part):
    # Neural Network architecture

    model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_part.shape[1],)),
    BatchNormalization(),  # Normalize the input to the next layer
    Dense(256, activation='relu'),
    Dropout(0.2),  # Dropout for regularization
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='selu'),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for predicting a single continuous value
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=0.5)
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=500, restore_best_weights=True)

    # Train the model
    model.fit(X_train_part, y_train_part, epochs=1000, batch_size=256, verbose=1, callbacks=[early_stopping], validation_split=0.1)

    return model

# Split the training data into parts
num_parts = 12  # Adjust this based on the number of CPU cores available
X_train_parts = np.array_split(X_train, num_parts)
y_train_parts = np.array_split(y_train, num_parts)

# Parallel training
models = ray.get([train_neural_network.remote(X_part, y_part) for X_part, y_part in zip(X_train_parts, y_train_parts)])

# Initialize an empty array to store predictions from each sub-model
all_predictions = np.zeros((X_test.shape[0], len(models)))  # Shape: number of test instances x number of models

# Collect predictions from each sub-model
for i, model in enumerate(models):
    # Predict using the current sub-model
    y_pred_submodel = model.predict(X_test)
    
    # Store predictions in the corresponding column of all_predictions. ravel make data 1D
    all_predictions[:, i] = y_pred_submodel.ravel()

# Aggregate predictions by averaging over the second dimension (models)
y_pred_combined = np.mean(all_predictions, axis=1)

# Ensure y_test is a 1D array for comparison
y_test_1d = y_test.ravel()

# Evaluate the aggregated predictions
mse = (np.mean((y_test_1d - y_pred_combined)**2))
print(f"Mean Squared Error(SQRT): {np.sqrt(mse)}")

# Check dimensions
print("Dimension of y_test_1d:", y_test_1d.shape)
print("Dimension of y_pred_combined:", y_pred_combined.shape)

# Save predictions to a file (optional)
predictions = pd.DataFrame({'Actual': y_test_1d, 'Predicted': y_pred_combined})
predictions.to_csv('house_price_predictions_nn.csv', index=False)



2024-02-04 22:23:48,462	INFO worker.py:1724 -- Started a local Ray instance.
/Users/behnamkheyraddinimousavi/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


(train_neural_network pid=8795) Epoch 1/1000


(train_neural_network pid=8797) WARNING:absl:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.


5/6 [========================>.....] - ETA: 0s - loss: 40270405632.0000
(train_neural_network pid=8795) Epoch 34/1000 [repeated 396x across cluster]
6/6 [==============================] - ETA: 0s - loss: 26609145856.0000
(train_neural_network pid=8795) Epoch 69/1000 [repeated 421x across cluster]
6/6 [==============================] - ETA: 0s - loss: 21653903360.0000
(train_neural_network pid=8795) Epoch 104/1000 [repeated 419x across cluster]
2/6 [=========>....................] - ETA: 0s - loss: 28573421568.0000
(train_neural_network pid=8792) Epoch 136/1000 [repeated 417x across cluster]
6/6 [==============================] - ETA: 0s - loss: 15501393920.0000
(train_neural_network pid=8795) Epoch 174/1000 [repeated 416x across cluster]
6/6 [==============================] - 0s 26ms/step - loss: 9805779968.0000 - val_loss: 26715627520.0000
(train_neural_network pid=8795) Epoch 209/1000 [repeated 413x across cluster]
6/6 [==============================] - 0s 26ms/step - loss: 100319651

/var/folders/rx/m8prdkhx6312x5qsk_xvrvrw0000gn/T/ipykernel_6896/793905620.py:85: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test_1d = y_test.ravel()


In [24]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
import ray

# Ensure TensorFlow is set to use GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Load the data
house_price_data = pd.read_csv('House_Price_Data.csv')
    # Convert date string to a numeric feature, e.g., days since a reference date
    # Here, assuming 'date' is the column with date strings like '20140527T000000'
house_price_data['date'] = pd.to_datetime(house_price_data['date']).map(pd.Timestamp.toordinal)

# Create the feature matrix X and target variable y
X = house_price_data[['Area', 'Bedrooms', 'Location', 'Condition', 'Ybuilt', 'Grade', 'Floors', 'View', 'Waterfront', 'sqft_living', 'yr_renovated', 'sqft_basement', 'sqft_above', 'sqft_living15', 'sqft_lot15', 'date']]
y = house_price_data['Price']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Shutdown Ray
ray.shutdown()

# Initialize Ray, specifying the number of GPUs to use
ray.init(num_gpus=40)

@ray.remote(num_gpus=1)  # Specify that each task should use one GPU
def train_neural_network(X_train_part, y_train_part):
    # Set TensorFlow to use a specific GPU, preventing TensorFlow from consuming all GPU memory
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Restrict TensorFlow to only use the GPU allocated by Ray
            tf.config.experimental.set_visible_devices(gpus[ray.get_gpu_ids()[0]], 'GPU')
            # Enable memory growth for the allocated GPU to prevent TensorFlow from consuming all GPU memory
            tf.config.experimental.set_memory_growth(gpus[ray.get_gpu_ids()[0]], True)
        except RuntimeError as e:
            # Handle potential errors
            print(e)

    # Neural Network architecture
    model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_part.shape[1],)),
    BatchNormalization(),  # Normalize the input to the next layer
    Dense(256, activation='relu'),
    Dropout(0.2),  # Dropout for regularization
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='selu'),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for predicting a single continuous value
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=500, restore_best_weights=True)

    # Train the model
    model.fit(X_train_part, y_train_part, epochs=1000, batch_size=256, verbose=1, callbacks=[early_stopping], validation_split=0.1)

    return model

# Split the training data into parts equal to the number of GPUs
X_train_parts = np.array_split(X_train, 40)
y_train_parts = np.array_split(y_train, 40)

# Parallel training across GPUs
models = ray.get([train_neural_network.remote(X_part, y_part) for X_part, y_part in zip(X_train_parts, y_train_parts)])

# Initialize an empty array to store predictions from each sub-model
all_predictions = np.zeros((X_test.shape[0], len(models)))

# Collect predictions from each sub-model
for i, model in enumerate(models):
    # Predict using the current sub-model
    y_pred_submodel = model.predict(X_test)
    
    # Store predictions in the corresponding column of all_predictions
    all_predictions[:, i] = y_pred_submodel.ravel()

# Aggregate predictions by averaging over the second dimension (models)
y_pred_combined = np.mean(all_predictions, axis=1)

# Ensure y_test is a 1D array for comparison
y_test_1d = y_test.ravel()

# Evaluate the aggregated predictions
mse = (np.mean((y_test_1d - y_pred_combined)**2))
print(f"Mean Squared Error(SQRT): {np.sqrt(mse)}")

# Check dimensions
print("Dimension of y_test_1d:", y_test_1d.shape)
print("Dimension of y_pred_combined:", y_pred_combined.shape)



2024-02-04 22:20:01,657	INFO worker.py:1724 -- Started a local Ray instance.
/Users/behnamkheyraddinimousavi/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


(train_neural_network pid=8666) Epoch 1/1000
2/2 [==============================] - ETA: 0s - loss: 24801021952.0000
(train_neural_network pid=8671) Epoch 66/1000 [repeated 1054x across cluster]
2/2 [==============================] - ETA: 0s - loss: 14571379712.0000
(train_neural_network pid=8666) Epoch 136/1000 [repeated 1122x across cluster]
2/2 [==============================] - 0s 94ms/step - loss: 4493373952.0000 - val_loss: 37879365632.0000
(train_neural_network pid=8662) Epoch 204/1000 [repeated 1106x across cluster]
2/2 [==============================] - 0s 82ms/step - loss: 4749178880.0000 - val_loss: 166098468864.0000
(train_neural_network pid=8658) Epoch 272/1000 [repeated 1107x across cluster]
2/2 [==============================] - 0s 49ms/step - loss: 3238518784.0000 - val_loss: 125776437248.0000 [repeated 885x across cluster]
(train_neural_network pid=8666) Epoch 347/1000 [repeated 1096x across cluster]
2/2 [==============================] - 0s 53ms/step - loss: 267884825

/var/folders/rx/m8prdkhx6312x5qsk_xvrvrw0000gn/T/ipykernel_6896/820526241.py:99: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test_1d = y_test.ravel()


In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import ray

def configure_tensorflow_gpus():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

@ray.remote
def preprocess_data(chunk, mean, scale):
    return (chunk - mean) / scale

@ray.remote(num_gpus=1)
def train_neural_network(X_train_part, y_train_part):
    configure_tensorflow_gpus()
    
    model = Sequential([
        Dense(256, activation='relu', input_shape=(X_train_part.shape[1],)),
        BatchNormalization(),
        Dense(256, activation='relu'),
        Dropout(0.2),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(64, activation='selu'),
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    model.fit(X_train_part, y_train_part, epochs=100, batch_size=128, verbose=1, callbacks=[early_stopping], validation_split=0.1)
    
    return model

def main():
    ray.shutdown()
    ray.init(num_cpus=12, num_gpus=4)

    house_price_data = pd.read_csv('House_Price_Data.csv')
    
    # Convert date string to a numeric feature, e.g., days since a reference date
    # Here, assuming 'date' is the column with date strings like '20140527T000000'
    house_price_data['date'] = pd.to_datetime(house_price_data['date']).map(pd.Timestamp.toordinal)
    
    X = house_price_data.drop(columns=['Price'])
    y = house_price_data['Price']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler().fit(X_train)
    mean, scale = scaler.mean_, scaler.scale_
    
    data_chunks = np.array_split(X_train, 12)
    processed_chunks = ray.get([preprocess_data.remote(chunk, mean, scale) for chunk in data_chunks])
    X_train_processed = np.concatenate(processed_chunks)
    
    X_test_processed = (X_test - mean) / scale

    X_train_parts = np.array_split(X_train_processed, 4)
    y_train_parts = np.array_split(y_train, 4)

    models = ray.get([train_neural_network.remote(X_part, y_part) for X_part, y_part in zip(X_train_parts, y_train_parts)])

    all_predictions = np.array([model.predict(X_test_processed) for model in models]).squeeze().T
    y_pred_combined = np.mean(all_predictions, axis=1)
    
    rmse = np.sqrt(np.mean((y_test - y_pred_combined) ** 2))
    print(f"Root Mean Squared Error: {rmse}")

    comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_combined})
    comparison_df.to_csv('model_predictions_comparison.csv', index=False)
    print("Saved the predictions comparison to 'model_predictions_comparison.csv'.")

if __name__ == "__main__":
    main()

2024-02-04 22:12:29,024	INFO worker.py:1724 -- Started a local Ray instance.
/Users/behnamkheyraddinimousavi/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/Users/behnamkheyraddinimousavi/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


(train_neural_network pid=8609) Epoch 1/100
26/31 [========================>.....] - ETA: 0s - loss: 18924023808.0000
(train_neural_network pid=8610) Epoch 21/100 [repeated 83x across cluster]
136/136 [==============================] - 0s 1ms/step
Root Mean Squared Error: 142930.9545669838
Saved the predictions comparison to 'model_predictions_comparison.csv'.


In [3]:
### X matrix will have three elements of area, bedrooms and location to predict y
### the relation between columns of X and non-linear effects will be considered
    from tensorflow.keras.layers import Dense, Dropout
    from tensorflow.keras.callbacks import EarlyStopping
    from tensorflow.keras.models import Sequential
    import pandas as pd
    import numpy as np
    import tensorflow as tf
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler


    #Load the data
    house_price_data = pd.read_csv('House_Price_Data.csv')
    house_price_data['date'] = house_price_data['date'].str[:6]

    # Create the feature matrix X and target variable y
    X = house_price_data[['Area', 'Bedrooms', 'Location', 'Condition', 'Ybuilt', 'Grade', 'Floors', 'View', 'Waterfront', 'sqft_living', 'yr_renovated', 'sqft_basement', 'sqft_above', 'sqft_living15', 'sqft_lot15', 'date']]
    y = house_price_data['Price']



    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Neural Network architecture
    model = tf.keras.Sequential([
    
    tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),#input_shape=(X_train.shape[1], trains based on the 3 columns
    tf.keras.layers.Dense(1028, activation='relu'),   
    tf.keras.layers.Dense(1028, activation='relu', input_shape=(X_train.shape[1],)), 
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu',input_shape=(X_train.shape[1],)),  # Output layer (regression, so one output neuron)
    tf.keras.layers.Dense(1,input_shape=(X_train.shape[1],))
    ])
    #Relu an activation function that introduces non-linearity to the model by allowing the model to learn complex patterns in the data. 
    #ReLU activation is commonly used and helps the network learn and model more complex relationships within the data.
    #in tf.keras.layers.Dense(64.... first layer there is 64 neurons in one layer each connected to all the inputs







    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    #Here are the key features of the Adam optimizer:
    ## Adaptive Learning Rate: Adam adjusts the learning rates for each parameter
    ## Momentum: It utilizes the concept of momentum to help accelerate convergence
    ## Bias Correction: Adam performs bias correction during the initial time steps to correct the estimates of the first and second moments, preventing large moments at first cycles
    ## RELU is for convergence and SELU for self normalizing compared to other neurons

    early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

    # Train the model
    model.fit(X_train, y_train, epochs=500, batch_size=400, verbose=1)

    # Evaluate the model
    mse = model.evaluate(X_test, y_test)
    print(f"Mean Squared Error(SQRT): {np.sqrt(mse)}")

    # Make predictions
    y_pred = model.predict(X_test)

    # Save predictions to a file (optional)
    predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})

    predictions.to_csv('house_price_predictions_nn.csv', index=False)


Epoch 1/500
44/44 [==============================] - 2s 30ms/step - loss: 403333152768.0000
Epoch 2/500
44/44 [==============================] - 1s 29ms/step - loss: 103021379584.0000
Epoch 3/500
44/44 [==============================] - 1s 30ms/step - loss: 51232645120.0000
Epoch 4/500
44/44 [==============================] - 1s 30ms/step - loss: 44309000192.0000
Epoch 5/500
44/44 [==============================] - 1s 31ms/step - loss: 41509425152.0000
Epoch 6/500
44/44 [==============================] - 1s 31ms/step - loss: 40141541376.0000
Epoch 7/500
44/44 [==============================] - 1s 34ms/step - loss: 39509606400.0000
Epoch 8/500
44/44 [==============================] - 1s 31ms/step - loss: 38707552256.0000
Epoch 9/500
44/44 [==============================] - 1s 31ms/step - loss: 38118690816.0000
Epoch 10/500
44/44 [==============================] - 1s 32ms/step - loss: 37684371456.0000
Epoch 11/500
44/44 [==============================] - 1s 33ms/step - loss: 37387497472.

In [3]:
### X matrix will have three elements of area, bedrooms and location to predict y
### the relation between columns of X and non-linear effects will be considered
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Load the data
house_price_data = pd.read_csv('House_Price_Data.csv')
house_price_data['date'] = house_price_data['date'].str[:6]

# Create the feature matrix X and target variable y
X = house_price_data[['Area', 'Bedrooms', 'Location', 'Condition', 'Ybuilt', 'Grade', 'Floors', 'View', 'Waterfront', 'sqft_living', 'yr_renovated', 'sqft_basement', 'sqft_above', 'sqft_living15', 'sqft_lot15', 'date']]
y = house_price_data['Price']



# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Neural Network architecture
model = tf.keras.Sequential([
    
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),#input_shape=(X_train.shape[1], trains based on the 3 columns
    tf.keras.layers.Dense(1028, activation='relu'),   
    tf.keras.layers.Dense(1028, activation='relu', input_shape=(X_train.shape[1],)), 
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu',input_shape=(X_train.shape[1],)),  # Output layer (regression, so one output neuron)
    tf.keras.layers.Dense(1,input_shape=(X_train.shape[1],))
])
#Relu an activation function that introduces non-linearity to the model by allowing the model to learn complex patterns in the data. 
#ReLU activation is commonly used and helps the network learn and model more complex relationships within the data.
#in    tf.keras.layers.Dense(64.... first layer there is 64 neurons in one layer each connected to all the inputs







# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
#Here are the key features of the Adam optimizer:
## Adaptive Learning Rate: Adam adjusts the learning rates for each parameter
## Momentum: It utilizes the concept of momentum to help accelerate convergence
## Bias Correction: Adam performs bias correction during the initial time steps to correct the estimates of the first and second moments, preventing large moments at first cycles
## RELU is for convergence and SELU for self normalizing compared to other neurons

early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=1500, batch_size=400)

# Evaluate the model
mse = model.evaluate(X_test, y_test)
print(f"Mean Squared Error(SQRT): {np.sqrt(mse)}")

# Make predictions
y_pred = model.predict(X_test)

# Save predictions to a file (optional)
predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
predictions.to_csv('house_price_predictions_nn.csv', index=False)

Epoch 1/1500
44/44 [==============================] - 2s 20ms/step - loss: 412619472896.0000
Epoch 2/1500
44/44 [==============================] - 1s 20ms/step - loss: 157593976832.0000
Epoch 3/1500
44/44 [==============================] - 1s 21ms/step - loss: 54675525632.0000
Epoch 4/1500
44/44 [==============================] - 1s 20ms/step - loss: 46768021504.0000
Epoch 5/1500
44/44 [==============================] - 1s 20ms/step - loss: 43292246016.0000
Epoch 6/1500
44/44 [==============================] - 1s 21ms/step - loss: 41357467648.0000
Epoch 7/1500
44/44 [==============================] - 1s 21ms/step - loss: 40466055168.0000
Epoch 8/1500
44/44 [==============================] - 1s 21ms/step - loss: 39584485376.0000
Epoch 9/1500
44/44 [==============================] - 1s 28ms/step - loss: 38907547648.0000
Epoch 10/1500
44/44 [==============================] - 1s 22ms/step - loss: 38429044736.0000
Epoch 11/1500
44/44 [==============================] - 1s 23ms/step - loss: 3

In [3]:
### X matrix will have three elements of area, bedrooms and location to predict y
### the relation between columns of X and non-linear effects will be considered
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Load the data
house_price_data = pd.read_csv('House_Price_Data.csv')
house_price_data['date'] = house_price_data['date'].str[:6]

# Create the feature matrix X and target variable y
X = house_price_data[['Area', 'Bedrooms', 'Location', 'Condition', 'Ybuilt', 'Grade', 'Floors', 'View', 'Waterfront', 'sqft_living', 'yr_renovated', 'sqft_basement', 'sqft_above', 'sqft_living15', 'sqft_lot15', 'date']]
y = house_price_data['Price']
X['Location'] =  (X['Location']-98000) 


# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Neural Network architecture
model = tf.keras.Sequential([
    
    tf.keras.layers.Dense(528, activation='relu', input_shape=(X_train.shape[1],)),#input_shape=(X_train.shape[1], trains based on the 3 columns
    tf.keras.layers.Dense(1028, activation='relu'),   
    tf.keras.layers.Dense(1028, activation='relu', input_shape=(X_train.shape[1],)), 
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu',input_shape=(X_train.shape[1],)),  # Output layer (regression, so one output neuron)
    tf.keras.layers.Dense(1,input_shape=(X_train.shape[1],))
])
#Relu an activation function that introduces non-linearity to the model by allowing the model to learn complex patterns in the data. 
#ReLU activation is commonly used and helps the network learn and model more complex relationships within the data.
#in    tf.keras.layers.Dense(64.... first layer there is 64 neurons in one layer each connected to all the inputs







# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
#Here are the key features of the Adam optimizer:
## Adaptive Learning Rate: Adam adjusts the learning rates for each parameter
## Momentum: It utilizes the concept of momentum to help accelerate convergence
## Bias Correction: Adam performs bias correction during the initial time steps to correct the estimates of the first and second moments, preventing large moments at first cycles
## RELU is for convergence and SELU for self normalizing compared to other neurons

early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=500, batch_size=528, verbose=1)

# Evaluate the model
mse = model.evaluate(X_test, y_test)
print(f"Mean Squared Error(SQRT): {np.sqrt(mse)}")

# Make predictions
y_pred = model.predict(X_test)

# Save predictions to a file (optional)
predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
predictions.to_csv('house_price_predictions_nn.csv', index=False)

/var/folders/3b/83lsl6j97g974xk6mcgg4tmr0000gn/T/ipykernel_6845/2945394760.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Location'] =  (X['Location']-98000)


Epoch 1/500
33/33 [==============================] - 2s 29ms/step - loss: 417133330432.0000
Epoch 2/500
33/33 [==============================] - 1s 31ms/step - loss: 231269269504.0000
Epoch 3/500
33/33 [==============================] - 1s 31ms/step - loss: 61594103808.0000
Epoch 4/500
33/33 [==============================] - 1s 32ms/step - loss: 49460183040.0000
Epoch 5/500
33/33 [==============================] - 1s 33ms/step - loss: 44489068544.0000
Epoch 6/500
33/33 [==============================] - 1s 34ms/step - loss: 42161930240.0000
Epoch 7/500
33/33 [==============================] - 1s 32ms/step - loss: 40773246976.0000
Epoch 8/500
33/33 [==============================] - 1s 38ms/step - loss: 39780503552.0000
Epoch 9/500
33/33 [==============================] - 1s 38ms/step - loss: 39248019456.0000
Epoch 10/500
33/33 [==============================] - 1s 34ms/step - loss: 38706167808.0000
Epoch 11/500
33/33 [==============================] - 1s 34ms/step - loss: 38370672640.